# 03.05 - Merge Demand, Weather, Calendar Data

## Imports & setup

In [1]:
import pathlib
import datetime
import dateutil
from os import PathLike
from typing import Union

#import simplegeneric
import pandas as pd
import numpy as np
from astral import Astral

import matplotlib.pyplot as plt
plt.style.use('grayscale')
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import palettable
%matplotlib inline


PROJECT_DIR = pathlib.Path.cwd().parent.resolve()

IMPUTED_DATA_DIR_DEMAND = PROJECT_DIR / 'data' / '03-imputed' / 'demand'
INTERIM_DATA_DIR_DEMAND = PROJECT_DIR / 'data' / '04-interim' / 'demand'
CALCULATED_FEATURES_DATA_DIR_CALENDAR = PROJECT_DIR / 'data' / '03-calculated-features' / 'calendar'
IMPUTED_DATA_DIR_WEATHER = PROJECT_DIR / 'data' / '03-imputed' / 'weather-toronto'
CLEAN_DATA_DIR = PROJECT_DIR / 'data' / '05-clean'

## Join Calendar features and Imputed Demand

In [2]:
cal_df = pd.read_csv(CALCULATED_FEATURES_DATA_DIR_CALENDAR / 'calendar.csv', index_col=0, parse_dates=True,
                       date_parser=dateutil.parser.parse)
cal_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 9 columns):
hour_of_day        222840 non-null int64
year               222840 non-null int64
month              222840 non-null int64
day_of_week        222840 non-null int64
day_of_year        222840 non-null int64
week_of_year       222840 non-null int64
quarter            222840 non-null int64
stat_hol           222840 non-null bool
day_light_hours    222840 non-null bool
dtypes: bool(2), int64(7)
memory usage: 14.0 MB


In [3]:
power_df = pd.read_csv(IMPUTED_DATA_DIR_DEMAND / 'demand.csv', index_col=0, parse_dates=True,
                      date_parser=dateutil.parser.parse)
power_df.rename(columns={'ont_demand': 'hourly_demand'}, inplace=True)

power_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 1 columns):
hourly_demand    222840 non-null float64
dtypes: float64(1)
memory usage: 3.4 MB


In [4]:
feat_df = pd.concat([cal_df, power_df], axis=1, sort=True)
feat_df.head()

,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1994-01-01 00:00:00,0,1994,1,5,1,52,1,True,False,14422.0
1994-01-01 01:00:00,1,1994,1,5,1,52,1,True,False,13845.0
1994-01-01 02:00:00,2,1994,1,5,1,52,1,True,False,13372.0
1994-01-01 03:00:00,3,1994,1,5,1,52,1,True,False,13025.0
1994-01-01 04:00:00,4,1994,1,5,1,52,1,True,False,12869.0


In [5]:
feat_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 10 columns):
hour_of_day        222840 non-null int64
year               222840 non-null int64
month              222840 non-null int64
day_of_week        222840 non-null int64
day_of_year        222840 non-null int64
week_of_year       222840 non-null int64
quarter            222840 non-null int64
stat_hol           222840 non-null bool
day_light_hours    222840 non-null bool
hourly_demand      222840 non-null float64
dtypes: bool(2), float64(1), int64(7)
memory usage: 15.7 MB


In [6]:
feat_df.to_csv(INTERIM_DATA_DIR_DEMAND / 'demand.csv')

## Join Demand, Demand Features, Calendar features, Weather Features

In [7]:
int_df = pd.read_csv(INTERIM_DATA_DIR_DEMAND / 'demand.csv', index_col=0, parse_dates=True,
                       date_parser=dateutil.parser.parse)
int_df.rename(columns ={'ont_demand': 'hourly_demand'}, inplace=True)
int_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 10 columns):
hour_of_day        222840 non-null int64
year               222840 non-null int64
month              222840 non-null int64
day_of_week        222840 non-null int64
day_of_year        222840 non-null int64
week_of_year       222840 non-null int64
quarter            222840 non-null int64
stat_hol           222840 non-null bool
day_light_hours    222840 non-null bool
hourly_demand      222840 non-null float64
dtypes: bool(2), float64(1), int64(7)
memory usage: 15.7 MB


In [8]:
int_df.head()

,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1994-01-01 00:00:00,0,1994,1,5,1,52,1,True,False,14422.0
1994-01-01 01:00:00,1,1994,1,5,1,52,1,True,False,13845.0
1994-01-01 02:00:00,2,1994,1,5,1,52,1,True,False,13372.0
1994-01-01 03:00:00,3,1994,1,5,1,52,1,True,False,13025.0
1994-01-01 04:00:00,4,1994,1,5,1,52,1,True,False,12869.0


In [9]:
int_df.tail()

,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
2019-06-03 19:00:00,19,2019,6,0,154,23,2,False,True,15518.0
2019-06-03 20:00:00,20,2019,6,0,154,23,2,False,True,15513.0
2019-06-03 21:00:00,21,2019,6,0,154,23,2,False,False,14664.0
2019-06-03 22:00:00,22,2019,6,0,154,23,2,False,False,13517.0
2019-06-03 23:00:00,23,2019,6,0,154,23,2,False,False,12464.0


In [10]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,
          'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,
          'hmdx': np.float64, 'wind_chill': np.float64, 'weather': np.object}
weather_df = pd.read_csv(IMPUTED_DATA_DIR_WEATHER / "weather_toronto.csv", parse_dates=True,
                 dtype=dtypes, index_col=0, infer_datetime_format=True)

weather_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 582433 entries, 1953-01-01 00:00:00 to 2019-06-11 23:00:00
Data columns (total 9 columns):
temp              582433 non-null float64
dew_point_temp    582433 non-null float64
rel_hum           582433 non-null float64
wind_speed        582433 non-null float64
visibility        582433 non-null float64
press             582433 non-null float64
hmdx              77410 non-null float64
wind_chill        143782 non-null float64
weather           552716 non-null object
dtypes: float64(8), object(1)
memory usage: 44.4+ MB


In [11]:
weather_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather
1953-01-01 00:00:00,-5.6,-6.7,93.0,10.0,25.0,99.44,NaN,-10.0,Cloudy
1953-01-01 01:00:00,-5.6,-9.4,71.0,3.0,25.0,99.46,NaN,-7.0,Cloudy
1953-01-01 02:00:00,-5.0,-8.9,72.0,0.0,19.3,99.43,NaN,NaN,Cloudy
1953-01-01 03:00:00,-5.0,-7.8,79.0,0.0,19.3,99.56,NaN,NaN,Cloudy
1953-01-01 04:00:00,-4.4,-7.2,80.0,0.0,19.3,99.55,NaN,NaN,Cloudy


In [12]:
weather_df.tail()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather
2019-06-11 19:00:00,19.4,2.5,32.0,16.0,24.1,99.88,NaN,NaN,Clear
2019-06-11 20:00:00,18.0,1.8,33.0,9.0,24.1,99.89,NaN,NaN,NaN
2019-06-11 21:00:00,17.0,2.7,38.0,13.0,24.1,99.89,NaN,NaN,NaN
2019-06-11 22:00:00,14.7,4.0,48.0,11.0,24.1,99.92,NaN,NaN,Clear
2019-06-11 23:00:00,14.0,4.4,52.0,12.0,24.1,99.94,NaN,NaN,NaN


In [13]:
clean_df = pd.concat([weather_df, int_df], axis=1, sort=True)
clean_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
1953-01-01 00:00:00,-5.6,-6.7,93.0,10.0,25.0,99.44,NaN,-10.0,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 01:00:00,-5.6,-9.4,71.0,3.0,25.0,99.46,NaN,-7.0,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 02:00:00,-5.0,-8.9,72.0,0.0,19.3,99.43,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 03:00:00,-5.0,-7.8,79.0,0.0,19.3,99.56,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1953-01-01 04:00:00,-4.4,-7.2,80.0,0.0,19.3,99.55,NaN,NaN,Cloudy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
clean_df.tail()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand
2019-06-11 19:00:00,19.4,2.5,32.0,16.0,24.1,99.88,NaN,NaN,Clear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-11 20:00:00,18.0,1.8,33.0,9.0,24.1,99.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-11 21:00:00,17.0,2.7,38.0,13.0,24.1,99.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-11 22:00:00,14.7,4.0,48.0,11.0,24.1,99.92,NaN,NaN,Clear,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-06-11 23:00:00,14.0,4.4,52.0,12.0,24.1,99.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 582433 entries, 1953-01-01 00:00:00 to 2019-06-11 23:00:00
Data columns (total 19 columns):
temp               582433 non-null float64
dew_point_temp     582433 non-null float64
rel_hum            582433 non-null float64
wind_speed         582433 non-null float64
visibility         582433 non-null float64
press              582433 non-null float64
hmdx               77410 non-null float64
wind_chill         143782 non-null float64
weather            552716 non-null object
hour_of_day        222840 non-null float64
year               222840 non-null float64
month              222840 non-null float64
day_of_week        222840 non-null float64
day_of_year        222840 non-null float64
week_of_year       222840 non-null float64
quarter            222840 non-null float64
stat_hol           222840 non-null object
day_light_hours    222840 non-null object
hourly_demand      222840 non-null float64
dtypes: float64(16), object(3)
memory usag

In [16]:
# Create a daily_peak column
# Peak Daily demand
clean_df = clean_df.join(feat_df.groupby(by=['year', 'day_of_year'])['hourly_demand'].max(),
                       on=['year', 'day_of_year'], rsuffix='_peak_in_day_in_year')
clean_df.rename(columns={'hourly_demand_peak_in_day_in_year': 'daily_peak'}, inplace=True)

In [17]:
clean_df.to_csv(CLEAN_DATA_DIR / 'oversized.csv')

In [18]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,
          'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,
          'hmdx': np.float64, 'wind_chill': np.float64, 'weather': np.object,
         'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,
         'day_of_week': np.float64, 'day_of_year': np.float64, 'week_of_year': np.float64,
          'quarter': np.float64, 'stat_hol': np.object, 'day_light_hours': np.object,
          'hourly_demand': np.float64, 'daily_peak': np.float64, 'weekly_peak': np.float64,
          'monthly_peak': np.float64, 'quarterly_peak': np.float64, 'annual_peak': np.float64,
          'hour_as_pct_of_daily_peak': np.float64, 'daily_peak_as_pct_of_weekly_peak': np.float64,
         'monthly_peak_as_pct_of_annual_peak': np.float64,
          'quarterly_peak_as_pct_of_annual_peak': np.float64, 'daily_peak_as_pct_of_annual_peak': np.float64}

oversized_df = pd.read_csv(CLEAN_DATA_DIR / "oversized.csv", parse_dates=True,
                 dtype=dtypes, index_col=0, infer_datetime_format=True)

oversized_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 582433 entries, 1953-01-01 00:00:00 to 2019-06-11 23:00:00
Data columns (total 20 columns):
temp               582433 non-null float64
dew_point_temp     582433 non-null float64
rel_hum            582433 non-null float64
wind_speed         582433 non-null float64
visibility         582433 non-null float64
press              582433 non-null float64
hmdx               77410 non-null float64
wind_chill         143782 non-null float64
weather            552716 non-null object
hour_of_day        222840 non-null float64
year               222840 non-null float64
month              222840 non-null float64
day_of_week        222840 non-null float64
day_of_year        222840 non-null float64
week_of_year       222840 non-null float64
quarter            222840 non-null float64
stat_hol           222840 non-null object
day_light_hours    222840 non-null object
hourly_demand      222840 non-null float64
daily_peak         222840 non-null float64

In [19]:
clean_df = oversized_df.dropna(subset=['hourly_demand'])

clean_df.to_csv(CLEAN_DATA_DIR / "clean.csv")

In [20]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 20 columns):
temp               222840 non-null float64
dew_point_temp     222840 non-null float64
rel_hum            222840 non-null float64
wind_speed         222840 non-null float64
visibility         222840 non-null float64
press              222840 non-null float64
hmdx               34086 non-null float64
wind_chill         50453 non-null float64
weather            193245 non-null object
hour_of_day        222840 non-null float64
year               222840 non-null float64
month              222840 non-null float64
day_of_week        222840 non-null float64
day_of_year        222840 non-null float64
week_of_year       222840 non-null float64
quarter            222840 non-null float64
stat_hol           222840 non-null object
day_light_hours    222840 non-null object
hourly_demand      222840 non-null float64
daily_peak         222840 non-null float64


In [21]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,
          'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,
          'hmdx': np.float64, 'wind_chill': np.float64, 'weather': np.object,
         'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,
         'day_of_week': np.float64, 'day_of_year': np.float64, 'week_of_year': np.float64,
          'quarter': np.float64, 'stat_hol': np.object, 'day_light_hours': np.object,
          'hourly_demand': np.float64, 'daily_peak': np.float64, 'weekly_peak': np.float64,
          'monthly_peak': np.float64, 'quarterly_peak': np.float64, 'annual_peak': np.float64,
          'hour_as_pct_of_daily_peak': np.float64, 'daily_peak_as_pct_of_weekly_peak': np.float64,
         'monthly_peak_as_pct_of_annual_peak': np.float64,
          'quarterly_peak_as_pct_of_annual_peak': np.float64, 'daily_peak_as_pct_of_annual_peak': np.float64}

df = pd.read_csv(CLEAN_DATA_DIR / "clean.csv", parse_dates=True,
                 dtype=dtypes, index_col=0, infer_datetime_format=True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 222840 entries, 1994-01-01 00:00:00 to 2019-06-03 23:00:00
Data columns (total 20 columns):
temp               222840 non-null float64
dew_point_temp     222840 non-null float64
rel_hum            222840 non-null float64
wind_speed         222840 non-null float64
visibility         222840 non-null float64
press              222840 non-null float64
hmdx               34086 non-null float64
wind_chill         50453 non-null float64
weather            193245 non-null object
hour_of_day        222840 non-null float64
year               222840 non-null float64
month              222840 non-null float64
day_of_week        222840 non-null float64
day_of_year        222840 non-null float64
week_of_year       222840 non-null float64
quarter            222840 non-null float64
stat_hol           222840 non-null object
day_light_hours    222840 non-null object
hourly_demand      222840 non-null float64
daily_peak         222840 non-null float64


In [23]:
clean_cut_df = clean_df.loc['1994': '2018']
clean_cut_df.to_csv(CLEAN_DATA_DIR / "clean-cut.csv")

In [24]:
dtypes = {'temp': np.float64, 'dew_point_temp':np.float64, 'rel_hum':np.float64,
          'wind_speed': np.float64, 'visibility': np.float64, 'press': np.float64,
          'hmdx': np.float64, 'wind_chill': np.float64, 'weather': np.object,
         'hour_of_day': np.float64, 'year': np.float64, 'month': np.float64,
         'day_of_week': np.float64, 'day_of_year': np.float64, 'week_of_year': np.float64,
          'quarter': np.float64, 'stat_hol': np.object, 'day_light_hours': np.object,
          'hourly_demand': np.float64, 'daily_peak': np.float64, 'weekly_peak': np.float64,
          'monthly_peak': np.float64, 'quarterly_peak': np.float64, 'annual_peak': np.float64,
          'hour_as_pct_of_daily_peak': np.float64, 'daily_peak_as_pct_of_weekly_peak': np.float64,
         'monthly_peak_as_pct_of_annual_peak': np.float64,
          'quarterly_peak_as_pct_of_annual_peak': np.float64, 'daily_peak_as_pct_of_annual_peak': np.float64}

clean_cut_df = pd.read_csv(CLEAN_DATA_DIR / "clean-cut.csv", parse_dates=True,
                 dtype=dtypes, index_col=0, infer_datetime_format=True)

clean_cut_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 219144 entries, 1994-01-01 00:00:00 to 2018-12-31 23:00:00
Data columns (total 20 columns):
temp               219144 non-null float64
dew_point_temp     219144 non-null float64
rel_hum            219144 non-null float64
wind_speed         219144 non-null float64
visibility         219144 non-null float64
press              219144 non-null float64
hmdx               34073 non-null float64
wind_chill         48918 non-null float64
weather            191377 non-null object
hour_of_day        219144 non-null float64
year               219144 non-null float64
month              219144 non-null float64
day_of_week        219144 non-null float64
day_of_year        219144 non-null float64
week_of_year       219144 non-null float64
quarter            219144 non-null float64
stat_hol           219144 non-null object
day_light_hours    219144 non-null object
hourly_demand      219144 non-null float64
daily_peak         219144 non-null float64


In [25]:
clean_cut_df.head()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
1994-01-01 00:00:00,-1.2,-3.8,83.0,15.0,19.3,99.91,NaN,-6.0,Cloudy,0.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,14422.0,16892.0
1994-01-01 01:00:00,-0.9,-3.0,86.0,20.0,16.1,99.91,NaN,-6.0,Cloudy,1.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13845.0,16892.0
1994-01-01 02:00:00,-0.7,-3.2,83.0,15.0,16.1,99.87,NaN,-5.0,Cloudy,2.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13372.0,16892.0
1994-01-01 03:00:00,-0.8,-2.4,89.0,15.0,12.9,99.81,NaN,-5.0,Cloudy,3.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,13025.0,16892.0
1994-01-01 04:00:00,-1.0,-3.3,84.0,19.0,16.1,99.77,NaN,-6.0,Mostly Cloudy,4.0,1994.0,1.0,5.0,1.0,52.0,1.0,True,False,12869.0,16892.0


In [26]:
clean_cut_df.tail()

,temp,dew_point_temp,rel_hum,wind_speed,visibility,press,hmdx,wind_chill,weather,hour_of_day,year,month,day_of_week,day_of_year,week_of_year,quarter,stat_hol,day_light_hours,hourly_demand,daily_peak
2018-12-31 19:00:00,2.5,1.0,90.0,19.0,9.7,98.51,NaN,NaN,"Rain,Fog",19.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,16195.0,17125.0
2018-12-31 20:00:00,2.9,1.6,92.0,21.0,9.7,98.17,NaN,NaN,"Rain,Fog",20.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,15668.0,17125.0
2018-12-31 21:00:00,3.7,2.6,93.0,21.0,9.7,97.98,NaN,NaN,"Rain,Fog",21.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,14987.0,17125.0
2018-12-31 22:00:00,4.0,3.0,93.0,18.0,6.4,97.61,NaN,NaN,"Rain,Fog",22.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,14560.0,17125.0
2018-12-31 23:00:00,4.3,3.4,94.0,12.0,9.7,97.37,NaN,NaN,"Rain,Fog",23.0,2018.0,12.0,0.0,365.0,1.0,4.0,False,False,13828.0,17125.0
